In [ ]:
# import the necessary packages
import cv2
import math
import numpy as np # matrix, array operations
from PIL import Image
import matplotlib.pyplot as plt # plotting tools
from google.colab.patches import cv2_imshow # cv2.imshow in colab
%matplotlib inline

# Read Underwater Images

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
image1 = cv2.imread("/content/gdrive/MyDrive/.../img1.jpg")
image2 = cv2.imread("/content/gdrive/MyDrive/.../img2.jpg")

 # 1. Efficient Image Dehazing with Boundary Constraint and Contextual Regularization

 references:
* https://openaccess.thecvf.com/content_iccv_2013/papers/Meng_Efficient_Image_Dehazing_2013_ICCV_paper.pdf
* https://github.com/Utkarsh-Deshmukh/Single-Image-Dehazing-Python
* https://utkarsh-deshmukh.github.io/Single-Image-Dehazing-Python/

In [ ]:
# required to restart the runtime
!pip install image_dehazer

In [ ]:
import image_dehazer

In [ ]:
plt.subplots(figsize=(8, 9))

plt.subplot(2,1,1)
plt.imshow(cv2.cvtColor(np.hstack((image_dehazer.remove_haze(image1.copy()), image1)), cv2.COLOR_RGB2BGR))
plt.title("corrected (left) - image1 - original (right)")

plt.subplot(2,1,2)
plt.imshow(cv2.cvtColor(np.hstack((image_dehazer.remove_haze(image2.copy()), image2)), cv2.COLOR_RGB2BGR))
plt.title("corrected (left) - image2 - original (right)")

# 2. Single Image Haze Removal Using Dark Channel Prior

references:

* Single Image Haze Removal Using Dark Channel Prior, Kaiming He, Jian Sun, and Xiaoou Tang", in CVPR 2009
* Guided Image Filtering, Kaiming He, Jian Sun, and Xiaoou Tang", in ECCV 2010
* https://github.com/He-Zhang/image_dehaze

In [ ]:
def DarkChannel(im,sz):
    b,g,r = cv2.split(im)
    dc = cv2.min(cv2.min(r,g),b)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(sz,sz))
    dark = cv2.erode(dc,kernel)
    return dark

def AtmLight(im,dark):
    [h,w] = im.shape[:2]
    imsz = h*w
    numpx = int(max(math.floor(imsz/1000),1))
    darkvec = dark.reshape(imsz);
    imvec = im.reshape(imsz,3);

    indices = darkvec.argsort();
    indices = indices[imsz-numpx::]

    atmsum = np.zeros([1,3])
    for ind in range(1,numpx):
       atmsum = atmsum + imvec[indices[ind]]

    A = atmsum / numpx
    return A

def TransmissionEstimate(im,A,sz):
    omega = 0.95;
    im3 = np.empty(im.shape,im.dtype)

    for ind in range(0,3):
        im3[:,:,ind] = im[:,:,ind]/A[0,ind]

    transmission = 1 - omega*DarkChannel(im3,sz)
    return transmission

def Guidedfilter(im,p,r,eps):
    mean_I = cv2.boxFilter(im,cv2.CV_64F,(r,r))
    mean_p = cv2.boxFilter(p, cv2.CV_64F,(r,r))
    mean_Ip = cv2.boxFilter(im*p,cv2.CV_64F,(r,r))
    cov_Ip = mean_Ip - mean_I*mean_p

    mean_II = cv2.boxFilter(im*im,cv2.CV_64F,(r,r))
    var_I   = mean_II - mean_I*mean_I

    a = cov_Ip/(var_I + eps)
    b = mean_p - a*mean_I

    mean_a = cv2.boxFilter(a,cv2.CV_64F,(r,r))
    mean_b = cv2.boxFilter(b,cv2.CV_64F,(r,r))

    q = mean_a*im + mean_b
    return q

def TransmissionRefine(im,et):
    gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
    gray = np.float64(gray)/255
    r = 60
    eps = 0.0001
    t = Guidedfilter(gray,et,r,eps)

    return t

def Recover(im,t,A,tx = 0.1):
    res = np.empty(im.shape,im.dtype)
    t = cv2.max(t,tx)

    for ind in range(0,3):
        res[:,:,ind] = (im[:,:,ind]-A[0,ind])/t + A[0,ind]

    return res

def img_dehaze(img_cpy):

  I = img_cpy.astype('float64')/255
  dark = DarkChannel(I,15)
  A = AtmLight(I,dark)
  te = TransmissionEstimate(I,A,15)
  t = TransmissionRefine(img_cpy,te)
  J = Recover(I,t,A,0.1)

  return (J*255)

In [ ]:
print("corrected (left) - image1 - original (right)")
cv2_imshow(np.hstack((img_dehaze(image1.copy()), image1)))

print("\ncorrected (left) - image2 - original (right)")
cv2_imshow(np.hstack((img_dehaze(image2.copy()), image2)))